In [ ]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore') # 경고 메세지 숨기기

from ast import literal_eval # 딕셔너리 형태의 문자열을 딕셔너리로 변경
from sklearn.feature_extraction.text import CountVectorizer # 단어 들의 카운트(출현 빈도(frequency))로 여러 문서들을 벡터화
from sklearn.metrics.pairwise import cosine_similarity # 코사인 유사도

In [ ]:
movies=pd.read_csv("./drive/MyDrive/data-files/total_tmdbmovielist_new.csv")
print(movies.shape)
movies.head(1)

(631376, 26)


,Column1,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,0,False,/hQ4pYsIbP22TMXOUdSfC2mjWrO0.jpg,NaN,0.0,"[{'id': 18, 'name': '드라마'}, {'id': 80, 'name':...",NaN,2.0,tt0094675,fi,Ariel,주인공 카스리넨은 광부다. 일하던 탄광이 폐광이 되며 도산을 하고 사장은 유일하게 ...,9.027,/ojDg0PGvs6R9xYFodRct2kdI6wC.jpg,"[{'id': 2303, 'logo_path': None, 'name': 'Vill...","[{'iso_3166_1': 'FI', 'name': 'Finland'}]",1988-10-21,0.0,73.0,"[{'english_name': 'German', 'iso_639_1': 'de',...",Released,NaN,아리엘,False,6.9,150.0


In [ ]:
# null 값 체크
movies.isnull().sum()

Column1                       0
adult                       526
backdrop_path            480785
belongs_to_collection    618974
budget                      850
genres                      882
homepage                 561238
id                          875
imdb_id                  234616
original_language           865
original_title              877
overview                 605946
popularity                 2102
poster_path              205812
production_companies       1998
production_countries       2016
release_date              60496
revenue                    2386
runtime                   55448
spoken_languages           2392
status                     2392
tagline                  623539
title                      2394
video                      2392
vote_average               2392
vote_count                 2392
dtype: int64

In [ ]:
movies1 = movies.dropna(subset=['id', 'title', 'genres', 'vote_average', 'vote_count'])

In [ ]:
# 유사도 측정시 데이터양이 많아서 오류가 발생하여 60만개 -> 30만개(0초과) -> 3만개(10초과) -> 만개(100초과)로 수정하여 작업진행함
movies2=movies1[movies1['vote_count']>100]
movies2.shape

(10931, 26)

In [ ]:
# 주요 컬럼 추출
movies3_df=movies2[['id', 'title', 'genres', 'vote_average', 'vote_count']]
movies3_df.head(5)

,id,title,genres,vote_average,vote_count
0,2.0,아리엘,"[{'id': 18, 'name': '드라마'}, {'id': 80, 'name':...",6.9,150.0
1,3.0,천국의 그림자,"[{'id': 18, 'name': '드라마'}, {'id': 35, 'name':...",7.2,149.0
2,5.0,포룸,"[{'id': 80, 'name': '범죄'}, {'id': 35, 'name': ...",5.7,2035.0
3,6.0,킬러 나이트,"[{'id': 28, 'name': '액션'}, {'id': 53, 'name': ...",6.5,215.0
6,11.0,스타워즈: 에피소드 4 새로운 희망,"[{'id': 12, 'name': '모험'}, {'id': 28, 'name': ...",8.2,16272.0


In [ ]:
# 컬럼 길이 100으로 세팅
pd.set_option('max_colwidth', 100)
movies3_df[['genres']][:1]

,genres
0,"[{'id': 18, 'name': '드라마'}, {'id': 80, 'name': '범죄'}, {'id': 35, 'name': '코미디'}]"


In [ ]:
# apply()에 literal_eval 함수를 적용해 문자열을 객체로 변경
movies3_df['genres']=movies3_df['genres'].apply(literal_eval)
movies3_df.head(1)

,id,title,genres,vote_average,vote_count
0,2.0,아리엘,"[{'id': 18, 'name': '드라마'}, {'id': 80, 'name': '범죄'}, {'id': 35, 'name': '코미디'}]",6.9,150.0


In [ ]:
# apply lambda를 이용하여 리스트 내 여러 개의 딕셔너리의 'name' 키 찾아 리스트 객체로 변환.
movies3_df['genres']=movies3_df['genres'].apply(lambda x : [ y['name'] for y in x])
movies3_df[['genres']][:1]

,genres
0,"[드라마, 범죄, 코미디]"


In [ ]:
movies3_df[['genres']]

,genres
0,"[드라마, 범죄, 코미디]"
1,"[드라마, 코미디]"
2,"[범죄, 코미디]"
3,"[액션, 스릴러, 범죄]"
6,"[모험, 액션, SF]"
...,...
613103,"[가족, 애니메이션, SF, 코미디]"
619456,"[스릴러, 드라마, 공포]"
622003,"[로맨스, 코미디]"
628475,"[애니메이션, 코미디, 가족, 판타지]"


In [ ]:
# CountVectorizer를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열로 변환.
movies3_df['genres_literal']=movies3_df['genres'].apply(lambda x : (' ').join(x))

# min_df는 너무 드물게로 나타나는 용어를 제거하는 데 사용. min_df = 0.01은 "문서의 1 % 미만"에 나타나는 용어를 무시한다. 
# ngram_range는 n-그램 범위.
count_vect=CountVectorizer(min_df=0, ngram_range=(1, 2))
genre_mat=count_vect.fit_transform(movies3_df['genres_literal'])
print(genre_mat.shape)

(10931, 328)


In [ ]:
# 유사도 측정
genre_sim=cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:1])

(10931, 10931)
[[1.         0.51639778 0.77459667 ... 0.25819889 0.16903085 0.25819889]]


In [ ]:
# [:, ::-1] axis = 1 기준으로 2차원 numpy 배열 뒤집기
genre_sim_sorted_ind=genre_sim.argsort()[:, ::-1]
print(genre_sim_sorted_ind[:1])

[[   0 8864 6836 ... 8912 6954 6051]]


In [ ]:
# 장르 콘텐츠 필터링 영화 추천
movies3_df[['title', 'vote_average', 'vote_count']].sort_values('vote_average', ascending=False)[:10]

,title,vote_average,vote_count
400193,브링 더 소울: 더 무비,9.3,296.0
503086,브레이크 더 사일런스: 더 무비,9.2,130.0
349747,번 더 스테이지: 더 무비,9.2,310.0
447673,BTS World Tour: Love Yourself - Japan Edition,9.2,262.0
522228,"마리셀라 에스코베도, 세 번의 죽음",9.0,176.0
187435,The Godfather Trilogy: 1901-1980,9.0,117.0
332013,극장판 바이올렛 에버가든,8.9,153.0
438737,빌리 아일리시: 조금 흐릿한 세상,8.7,202.0
241,쇼생크 탈출,8.7,20134.0
437004,"조제, 호랑이 그리고 물고기들",8.7,139.0


In [ ]:
# 가중평점 계산
# 가중 평점(Weighted Rating) = (v/(v+m)) * R + (m/(v+m)) * C
 # v : 영화에 평가를 매긴 횟수(movie_df의 'vote_count')
 # m : 평점을 부여하기 위한 최소 평가 수(movies_df['vote_count'].quantile(0.6) - 전체 투표 수에서 상위 60%의 횟수를 기준)
 # R : 영화의 평균 평점(movie_df의 'vote_average')
 # C : 전체 영화의 평균 평점(movie_df['vote_average'].mean())

percentile = 0.6
m = movies3_df['vote_count'].quantile(percentile)  # 평점을 부여하기 위한 최소 평가 수
C = movies3_df['vote_average'].mean()  # 전체 영화의 평균 평점

def weighted_vote_average(record):
  v = record['vote_count']  # 영화에 평가를 매긴 횟수
  R = record['vote_average']  # 영화의 평균 평점

  return ( (v/(v+m)) * R ) + ( (m/(m+v)) * C )  # 가중 평점 계산 식

movies3_df['weighted_vote'] = movies.apply(weighted_vote_average, axis=1)

movies3_df[['title', 'weighted_vote', 'vote_count']].sort_values('weighted_vote', ascending=False)[:10]

,title,weighted_vote,vote_count
241,쇼생크 탈출,8.649205,20134.0
195,대부,8.632740,15086.0
346,쉰들러 리스트,8.520279,12041.0
197,대부 2,8.495562,9076.0
194155,너의 이름은.,8.483311,8072.0
118,다크 나이트,8.464418,26213.0
568,펄프 픽션,8.457935,22098.0
8,포레스트 검프,8.456965,21589.0
88,반지의 제왕: 왕의 귀환,8.450184,18584.0
402,그린 마일,8.429835,13053.0


In [ ]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
  title_movie = df[df['title'] == title_name]
  title_index = title_movie.index.values

  # top_n의 2배에 해당하는 장르 유사성이 높은 인덱스 추출
  similar_indexes = sorted_ind[title_index, :(top_n*2)]
  # reshape(-1) 1차열 배열 반환
  similar_indexes = similar_indexes.reshape(-1)
  # 기준 영화 인덱스는 제외
  similar_indexes = similar_indexes[similar_indexes != title_index]

  # top_n의 2배에 해당하는 후보군에서 weighted_vote가 높은 순으로 top_n만큼 추출
  return df.iloc[similar_indexes].sort_values('weighted_vote', ascending=False)[:top_n]

similar_movies=find_sim_movie(movies3_df, genre_sim_sorted_ind, '쇼생크 탈출', 241)
similar_movies[['title', 'vote_count', 'weighted_vote']]

,title,vote_count,weighted_vote
526,인생은 아름다워,10587.0,8.414210
2075,모던 타임즈,2767.0,8.037972
259346,"러브, 사이먼",5167.0,7.966424
19081,그랜드 부다페스트 호텔,11646.0,7.941767
3920,키드,1464.0,7.787934
...,...,...,...
562171,굿바이 에이틴,149.0,6.584529
34754,하버드 졸업반,140.0,6.582871
4137,코쿤,934.0,6.582590
63397,Les petits princes,134.0,6.581739
